Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![04ACI](files/tables/tables_image4.JPG)

# Automated ML on Azure Databricks

In this example we use the scikit-learn's <a href="http://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwritten-digits-dataset" target="_blank">digit dataset</a> to showcase how you can use AutoML for a simple classification problem.

In this notebook you will learn how to:
1. Create Azure Machine Learning Workspace object and initialize your notebook directory to easily reload this object from a configuration file.
2. Create an `Experiment` in an existing `Workspace`.
3. Configure Automated ML using `AutoMLConfig`.
4. Train the model using Azure Databricks.
5. Explore the results.
6. Test the best fitted model.

Before running this notebook, please follow the <a href="https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks" target="_blank">readme for using Automated ML on Azure Databricks</a> for installing necessary libraries to your cluster.

We support installing AML SDK with Automated ML as library from GUI. When attaching a library follow <a href="https://docs.databricks.com/user-guide/libraries.html" target="_blank">this link</a> and add the below string as your PyPi package. You can select the option to attach the library to all clusters or just one cluster.

**azureml-sdk with automated ml**
* Source: Upload Python Egg or PyPi
* PyPi Name: `azureml-sdk[automl_databricks]`
* Select Install Library

### Check the Azure ML Core SDK Version to Validate Your Installation

In [6]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.3.0

## Initialize an Azure ML Workspace
### What is an Azure ML Workspace and Why Do I Need One?

An Azure ML workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an Azure ML workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.


### What do I Need?

To create or access an Azure ML workspace, you will need to import the Azure ML library and specify following information:
* A name for your workspace. You can choose one.
* Your subscription id. Use the `id` value from the `az account show` command output above.
* The resource group name. The resource group organizes Azure resources and provides a default region for the resources in the group. The resource group will be created if it doesn't exist. Resource groups can be created and viewed in the [Azure portal](https://portal.azure.com)
* Supported regions include `eastus2`, `eastus`,`westcentralus`, `southeastasia`, `westeurope`, `australiaeast`, `westus2`, `southcentralus`.

In [8]:
subscription_id = "04818311-1c9c-4d20-90d9-0df21c6e69c8" #you should be owner or contributor
resource_group = "automl" #you should be owner or contributor
workspace_name = "automl" #your workspace name
workspace_region = "westeurope" #your region

## Creating a Workspace
If you already have access to an Azure ML workspace you want to use, you can skip this cell.  Otherwise, this cell will create an Azure ML workspace for you in the specified subscription, provided you have the correct permissions for the given `subscription_id`.

This will fail when:
1. The workspace already exists.
2. You do not have permission to create a workspace in the resource group.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription.

If workspace creation fails for any reason other than already existing, please work with your IT administrator to provide you with the appropriate permissions or to provision the required resources.

**Note:** Creation of a new workspace can take several minutes.

In [10]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)
ws.get_details()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CWTEKUMWE to authenticate.
Interactive authentication successfully completed.
Out[7]: {'id': '/subscriptions/5ba98ffe-829c-4364-aade-e37427adda90/resourceGroups/hackathon/providers/Microsoft.MachineLearningServices/workspaces/AI-hackathon',
 'name': 'AI-hackathon',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': '27cc8895-d80b-4a09-80e6-989cd2470561',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-08-30T13:03:14.2700063+00:00',
 'containerRegistry': '/subscriptions/5ba98ffe-829c-4364-aade-e37427adda90/resourceGroups/hackathon/providers/Microsoft.ContainerRegistry/registries/aihackathon210173d3',
 'keyVault': '/subscriptions/5ba98ffe-829c-4364-aade-e37427adda90/resourcegroups/hackathon/providers/microsoft.keyvault/vaults/aihackathon6471936613',
 'applicationInsights': '/subscriptions/5ba98ffe-829c-4364-aade-e37427adda90/resourcegroups/hackathon/providers/microsoft.insights/components/aihackathon4764474794',
 'identityPrincipalId': 'e6028b76-ea9b-4935-9021-c273beb901c3',
 'identityTenantId': 'a7f1d862-decb-4991-9b4d-9aa7fed56118',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/5ba98ffe-829c-4364-aade-e37427adda90/resourcegroups/hackathon/providers/microsoft.storage/storageaccounts/aihackathon1827852733',
 'hbiWorkspace': False,
 'discoveryUrl': 'https://westeurope.experiments.azureml.net/discovery'}

## Configuring Your Local Environment
You can validate that you have access to the specified workspace and write a configuration file to the default configuration location, `./aml_config/config.json`.

In [12]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

## Create an Experiment

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [14]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [15]:
ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-titanic'
project_folder = './sample_projects/automl-titanic'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

,
SDK version,1.3.0
Subscription ID,5ba98ffe-829c-4364-aade-e37427adda90
Workspace Name,AI-hackathon
Resource Group,hackathon
Location,westeurope
Project Directory,./sample_projects/automl-titanic
Experiment Name,automl-titanic


## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases.

In [17]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on.

## Load Training Data Using DataPrep

In [19]:
import pandas as pd

X = pd.read_csv('/dbfs/titanic/train.csv')
y = X[['Survived']]
X = X.drop(['PassengerId', 'Survived', 'Cabin'], axis=1)
display(X)
#print(y)

Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C
3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,S
1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,S
3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,S
3,"Moran, Mr. James",male,null,0,0,330877,8.4583,Q
1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S
3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,S
3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S
2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C


In [20]:
#Automated ML requires a dataflow, which is different from dataframe.
#If your data is in a dataframe, please use read_pandas_dataframe to convert a dataframe to dataflow before usind dprep.

import azureml.dataprep as dprep
# You can use `auto_read_file` which intelligently figures out delimiters and datatypes of a file.
# The data referenced here was pulled from `sklearn.datasets.load_digits()`.
#simple_example_data_root = 'https://dprepdata.blob.core.windows.net/automl-notebook-data/'
#X_train = dprep.auto_read_file(simple_example_data_root + 'X.csv').skip(1)  # Remove the header row.
X_train = dprep.read_pandas_dataframe(X, temp_folder='/dbfs/tmp/X', overwrite_ok=1)

# You can also use `read_csv` and `to_*` transformations to read (with overridable delimiter)
# and convert column types manually.
# Here we read a comma delimited file and convert all columns to integers.
#y_train = dprep.read_csv(simple_example_data_root + 'y.csv').to_long(dprep.ColumnSelector(term='.*', use_regex = True))
y_train = dprep.read_pandas_dataframe(y, temp_folder='/dbfs/tmp/y', overwrite_ok=1)

## Review the Data Preparation Result
You can peek the result of a Dataflow at any range using skip(i) and head(j). Doing so evaluates only j records for all the steps in the Dataflow, which makes it fast even against large datasets.

In [22]:
X_train.skip(1).head(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.00,1,0,PC 17599,71.28,C
1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,S
2,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,S
3,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,S
4,3,"Moran, Mr. James",male,nan,0,0,330877,8.46,Q


In [23]:
y_train.skip(1).head(5)

,Survived
0,1
1,1
2,1
3,0
4,0


## Configure AutoML

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**spark_context**|Spark Context object. for Databricks, use spark_context=sc|
|**max_concurrent_iterations**|Maximum number of iterations to execute in parallel. This should be <= number of worker nodes in your Azure Databricks cluster.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [25]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'accuracy',
                             iteration_timeout_minutes = 10,
                             iterations = 30,
                             n_cross_validations = 10,
                             max_concurrent_iterations = 1, #change it based on number of worker nodes
                             verbosity = logging.INFO,
                             spark_context=sc, #databricks/spark related
                             X = X_train, 
                             y = y_train,
                             path = project_folder)

WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig

## Train the Models

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.
In this example, we specify `show_output = True` to print currently running iterations to the console. If you are running a lot of iterations, you can set it to False and visualize in the portal.

In [27]:
local_run = experiment.submit(automl_config, show_output = True) # for higher runs please use show_output=False and use the below

WARNING - Input of type `azureml.dataprep.Dataflow` for X will be deprecated.
WARNING - Input of type `azureml.dataprep.Dataflow` for y will be deprecated.
Running an experiment on spark cluster: automl-titanic.

Parent Run ID: AutoML_4436fb1d-299a-433a-8655-2e056553fcb0


****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 0 MaxAbsScaler LightGBM 0:00:54 0.8226 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 1 MaxAbsScaler SGD 0:00:59 0.8114 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 2 MaxAbsScaler SGD 0:00:51 0.8092 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 3 MaxAbsScaler ExtremeRandomTrees 0:00:50 0.6791 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 4 MaxAbsScaler SGD 0:00:54 0.8193 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 5 MaxAbsScaler RandomForest 0:00:54 0.6680 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 6 MaxAbsScaler SGD 0:00:50 0.7946 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 7 MaxAbsScaler RandomForest 0:01:02 0.7834 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 8 StandardScalerWrapper RandomForest 0:00:54 0.7251 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 9 MaxAbsScaler RandomForest 0:00:56 0.7968 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 10 MaxAbsScaler ExtremeRandomTrees 0:00:51 0.7912 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 11 MaxAbsScaler BernoulliNaiveBayes 0:00:48 0.7879 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 12 StandardScalerWrapper BernoulliNaiveBayes 0:00:55 0.6791 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 13 MaxAbsScaler BernoulliNaiveBayes 0:00:58 0.7800 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 14 MaxAbsScaler SGD 0:00:49 0.7912 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 15 MaxAbsScaler RandomForest 0:00:52 0.6924 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 16 MaxAbsScaler ExtremeRandomTrees 0:00:53 0.7924 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 17 MaxAbsScaler RandomForest 0:00:55 0.7283 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 18 MaxAbsScaler SGD 0:00:57 0.8103 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 19 MaxAbsScaler ExtremeRandomTrees 0:00:49 0.7677 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 20 MaxAbsScaler RandomForest 0:00:52 0.6171 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 21 MaxAbsScaler LightGBM 0:00:58 0.8058 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 22 MaxAbsScaler ExtremeRandomTrees 0:00:53 0.7879 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 23 SparseNormalizer LogisticRegression 0:00:57 0.8047 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 24 MaxAbsScaler LightGBM 0:00:53 0.7879 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 25 StandardScalerWrapper LightGBM 0:00:58 0.6161 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0]
 26 MaxAbsScaler ExtremeRandomTrees 0:00:48 0.7923 0.8226
ERROR: sparse matrix length is ambiguous; use getnnz() or shape[0

## Explore the Results

#### Portal URL for Monitoring Runs

The following will provide a link to the web interface to explore individual run details and status.

In [30]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

Your experiment in Azure Portal: AutoML_4436fb1d-299a-433a-8655-2e056553fcb0

#### Retrieve All Child Runs after the experiment has COMPLETED.
You can also use SDK methods to fetch all the child runs and see individual metrics that we log. This can take some time.

In [32]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
display(rundata)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0.8596816567387477,0.8379520662519353,0.8525298716076778,0.6505594501501714,0.84984058258848,0.6945760575722569,0.8330968924709932,0.8217249445712748,0.7021419217986333,0.8396167754900519,0.854201314520726,0.8371564917725871,0.6940961766258874,0.8312890858915521,0.8433313183780285,0.6898298713454623,0.8262383295046989,0.7733161214755674,0.8383497805939095,0.8455866524478954,0.6069322397309981,0.8571479659078397,0.849091799864287,0.8455463509496571,0.8511809343433704,0.8449469811467522,0.8335648398564883,0.8440859830750383,0.8637529416578833,0.864360191426673
0.8736213394305807,0.8569652852785454,0.8669922023188867,0.6897273243027987,0.8680918047197558,0.6977388984742854,0.8492002755606677,0.8190612826351581,0.7351431497145422,0.839192191096959,0.8642012450105284,0.8510597988469468,0.7011811476291341,0.8429576200785224,0.8600185021532074,0.7094630432309177,0.8215163271254253,0.7610316053123359,0.8569039761471693,0.8629728164388771,0.6389022429827883,0.8727355926814327,0.8597828198522135,0.8625403701054083,0.8627230428256814,0.777434056368366,0.8413714489223054,0.8583341219854708,0.8740385130322428,0.8784787695156335
0.8596816567387477,0.8379520662519354,0.8525298716076778,0.6505594501501714,0.84984058258848,0.6945760575722569,0.8330968924709932,0.8217249445712748,0.7021419217986334,0.8396167754900519,0.854201314520726,0.8371564917725871,0.6935746375670864,0.8312890858915521,0.8433313183780285,0.6898298713454623,0.8262383295046989,0.7733161214755675,0.8383497805939093,0.8455866524478954,0.6069322397309981,0.8571479659078397,0.849091799864287,0.8455463509496571,0.8511809343433704,0.8449469811467523,0.8335648398564883,0.8440859830750383,0.8637529416578833,0.864360191426673
0.8226342072409487,0.8114357053682897,0.8092134831460674,0.6790886392009987,0.8193008739076154,0.668027465667915,0.7946317103620475,0.7833957553058677,0.7250686641697878,0.7968414481897627,0.791223470661673,0.7878901373283396,0.6790886392009987,0.7800249687890137,0.791223470661673,0.6924094881398253,0.7923595505617979,0.7283395755305868,0.8103121098626715,0.767665418227216,0.6171036204744069,0.8057802746566791,0.787852684144819,0.8046691635455681,0.7878526841448188,0.6160799001248438,0.7923220973782772,0.8170661672908863,0.8394756554307117,0.8260049937578028
0.8498066589599356,0.8188391376372026,0.8496960850784412,0.6228267106327442,0.8449827598088671,0.6722761867127272,0.8252044435600382,0.8039892341773207,0.6619843169287654,0.8312521348060684,0.8504628766749706,0.8294392262572936,0.6478951736425045,0.824852153378244,0.8371619248267432,0.646268215806084,0.8003392981936814,0.7378050569161639,0.8292319564932951,0.8266144428249461,0.5816867011261379,0.8511637095578581,0.8429749465472977,0.8349232032997106,0.8454918623589167,0.8386558246615312,0.819184563679254,0.8360138755346481,0.8604218423834842,0.8594114617037321
0.8564750480554808,0.833249551018025,0.8542017569631002,0.6580840167900492,0.8507607325677528,0.6830199494075999,0.8319068981164623,0.8010386454205539,0.6953473679964872,0.8300061344450667,0.8501635642956782,0.839068696228076,0.6611807512129217,0.8316051339388542,0.8439658213569976,0.672137839084688,0.7982522098154847,0.7478197804352579,0.8394338275249581,0.8447699150665571,0.6206819054433297,0.8563846340311784,0.8462802568508373,0.847288744294222,0.8456096463808376,0.7812711217825401,0.8311481343140026,0.8396097888931273,0.8599540848652222,0.8623917287940577
0.8597109335499493,0.8302892188323059,0.8578786083223682,0.6508178610656614,0.853729706909031,0.6960544684483161,0.8354284159395912,0.8220347732261415,0.6879289236926553,0.8436304275003575,0.8600766994627602,0.8421335175989035,0.6751268695748177,0.8375011085906008,0.8475611837740298,0.6730653569660735,0.8187732128792046,0.7609798883007313,0.8392336719862554,0.8380950538104693,0.6114609922891843,0.8585502922604693,0.8538899597723585,0.8465552255346067,0.8541537676682951,0.8500832913138521,0.8363001810283688,0.8463401149

### Retrieve the Best Model after the experiment has COMPLETED.

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [34]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: automl-titanic,
Id: AutoML_4436fb1d-299a-433a-8655-2e056553fcb0_28,
Type: None,
Status: Completed)
Pipeline(memory=None,
 steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
 feature_sweeping_config=None, feature_sweeping_timeout=None,
 featurization_config=None, force_text_dnn=None,
 is_cross_validation=None, is_onnx_compatible=None, logger=None,
 obser... flatten_transform=None,
 weights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2]))])

#### Best Model Based on Any Other Metric after the above run is complete based on the child run
Show the run and the model that has the smallest `log_loss` value:

In [36]:
lookup_metric = "log_loss"
best_run, fitted_model = local_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: automl-titanic,
Id: AutoML_4436fb1d-299a-433a-8655-2e056553fcb0_29,
Type: None,
Status: Completed)
Pipeline(memory=None,
 steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
 feature_sweeping_config=None, feature_sweeping_timeout=None,
 featurization_config=None, force_text_dnn=None,
 is_cross_validation=None, is_onnx_compatible=None, logger=None,
 obser...7f45c47281d0>,
 solver='lbfgs', tol=0.0001, verbose=0),
 training_cv_folds=5))])

### Test the Best Fitted Model

#### Load Test Data - you can split the dataset beforehand & pass Train dataset to AutoML and use Test dataset to evaluate the best model.

In [38]:
X_test = pd.read_csv('/dbfs/titanic/test.csv')
y_test = X_test[['PassengerId']]
X_test = X_test.drop(['PassengerId', 'Cabin'], axis=1)

In [39]:
X_test.head(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.83,Q
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.00,S
2,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.69,Q
3,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.66,S
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.29,S


#### Testing Our Best Fitted Model
We will try to predict digits and see how our model works. This is just an example to show you.

In [41]:
y_test['Survived'] = fitted_model.predict(X_test)

In [42]:
display(y_test)

PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,1
899,0
900,1
901,0


In [43]:
y_test.to_csv('/dbfs/titanic/results-automl-sdk.csv', index=False)